In [1]:
# Check if the easydiffraction library is installed.
# If not, install it including the 'visualization' extras.
# This is needed, e.g., when running this as a notebook via Google Colab or
# Jupyter Notebook in an environment where the library is not pre-installed.
import builtins
import importlib.util

if hasattr(builtins, '__IPYTHON__'):
    if importlib.util.find_spec('easydiffraction') is None:
        !pip install 'easydiffraction[visualization]'


# Structure Refinement: HS, HRPT

This example demonstrates a Rietveld refinement of HS crystal structure
using constant wavelength neutron powder diffraction data from HRPT at PSI.

## Import Library

In [2]:
from easydiffraction import Experiment
from easydiffraction import Project
from easydiffraction import SampleModel
from easydiffraction import download_from_repository

## Define Sample Model

This section shows how to add sample models and modify their parameters.

#### Create Sample Model

In [3]:
model = SampleModel('hs')

#### Set Space Group

In [4]:
model.space_group.name_h_m = 'R -3 m'
model.space_group.it_coordinate_system_code = 'h'

#### Set Unit Cell

In [5]:
model.cell.length_a = 6.9
model.cell.length_c = 14.1

#### Set Atom Sites

In [6]:
model.atom_sites.add('Zn', 'Zn', 0, 0, 0.5, wyckoff_letter='b', b_iso=0.5)
model.atom_sites.add('Cu', 'Cu', 0.5, 0, 0, wyckoff_letter='e', b_iso=0.5)
model.atom_sites.add('O', 'O', 0.21, -0.21, 0.06, wyckoff_letter='h', b_iso=0.5)
model.atom_sites.add('Cl', 'Cl', 0, 0, 0.197, wyckoff_letter='c', b_iso=0.5)
model.atom_sites.add('H', '2H', 0.13, -0.13, 0.08, wyckoff_letter='h', b_iso=0.5)

#### Symmetry constraints

Show CIF output before applying symmetry constraints.

In [7]:
model.show_as_cif()


Sample model 🧩 'hs' as cif


data_hs
""
_space_group.IT_coordinate_system_code h
"_space_group.name_H-M_alt ""R -3 m"""
""
_cell.angle_alpha 90.0
_cell.angle_beta 90.0
_cell.angle_gamma 90.0
_cell.length_a 6.9
_cell.length_b 10.0
_cell.length_c 14.1


Apply symmetry constraints.

In [8]:
model.apply_symmetry_constraints()

Show CIF output after applying symmetry constraints.

In [9]:
model.show_as_cif()


Sample model 🧩 'hs' as cif


data_hs
""
_space_group.IT_coordinate_system_code h
"_space_group.name_H-M_alt ""R -3 m"""
""
_cell.angle_alpha 90
_cell.angle_beta 90
_cell.angle_gamma 120
_cell.length_a 6.9
_cell.length_b 6.9
_cell.length_c 14.1


## Define Experiment

This section shows how to add experiments, configure their parameters, and
link the sample models defined in the previous step.

#### Download Measured Data

In [10]:
download_from_repository('hrpt_hs.xye', destination='data')


⚠️ Warning
File 'data/hrpt_hs.xye' already exists and will not be overwritten.


#### Create Experiment

In [11]:
expt = Experiment(name='hrpt', data_path='data/hrpt_hs.xye')


Data loaded successfully
Experiment 🔬 'hrpt'. Number of data points: 3220


#### Set Instrument

In [12]:
expt.instrument.setup_wavelength = 1.89
expt.instrument.calib_twotheta_offset = 0.0

#### Set Peak Profile

In [13]:
expt.peak.broad_gauss_u = 0.1
expt.peak.broad_gauss_v = -0.2
expt.peak.broad_gauss_w = 0.2
expt.peak.broad_lorentz_x = 0.0
expt.peak.broad_lorentz_y = 0

#### Set Background

In [14]:
expt.background.add(x=4.4196, y=500)
expt.background.add(x=6.6207, y=500)
expt.background.add(x=10.4918, y=500)
expt.background.add(x=15.4634, y=500)
expt.background.add(x=45.6041, y=500)
expt.background.add(x=74.6844, y=500)
expt.background.add(x=103.4187, y=500)
expt.background.add(x=121.6311, y=500)
expt.background.add(x=159.4116, y=500)

#### Set Linked Phases

In [15]:
expt.linked_phases.add('hs', scale=0.5)

## Define Project

The project object is used to manage the sample model, experiment, and
analysis.

#### Create Project

In [16]:
project = Project()

#### Set Plotting Engine

In [17]:
project.plotter.engine = 'plotly'


Current plotter changed to
plotly


#### Add Sample Model

In [18]:
project.sample_models.add(model)

#### Add Experiment

In [19]:
project.experiments.add(expt)

## Perform Analysis

This section shows the analysis process, including how to set up
calculation and fitting engines.

#### Set Calculator

In [20]:
project.analysis.current_calculator = 'cryspy'


Current calculator changed to
cryspy


#### Set Minimizer

In [21]:
project.analysis.current_minimizer = 'lmfit (leastsq)'


Current minimizer changed to
lmfit (leastsq)


#### Plot Measured vs Calculated

In [22]:
project.plot_meas_vs_calc(expt_name='hrpt', show_residual=True)

In [23]:
project.plot_meas_vs_calc(expt_name='hrpt', x_min=48, x_max=51, show_residual=True)

### Perform Fit 1/5

Set parameters to be refined.

In [24]:
model.cell.length_a.free = True
model.cell.length_c.free = True

expt.linked_phases['hs'].scale.free = True
expt.instrument.calib_twotheta_offset.free = True

Show free parameters after selection.

In [25]:
project.analysis.show_free_params()


Free parameters for both sample models (🧩 data blocks) and experiments (🔬 data blocks)


,datablock,category,entry,parameter,value,uncertainty,min,max,units
1,hs,cell,,length_a,6.9000,,,,Å
2,hs,cell,,length_c,14.1000,,,,Å
3,hrpt,instrument,,twotheta_offset,0.0000,,,,deg
4,hrpt,linked_phases,hs,scale,0.5000,,,,


#### Run Fitting

In [26]:
project.analysis.fit()


Using experiment 🔬 'hrpt' for 'single' fitting
🚀 Starting fit process with 'lmfit (leastsq)'...
📈 Goodness-of-fit (reduced χ²) change:


iteration,χ²,improvement [%]
1,576.50,
8,122.02,78.8% ↓
13,115.19,5.6% ↓
18,109.86,4.6% ↓
23,106.68,2.9% ↓
28,104.60,2.0% ↓
33,102.87,1.7% ↓
38,101.13,1.7% ↓
43,99.20,1.9% ↓
48,96.86,2.4% ↓


🏆 Best goodness-of-fit (reduced χ²) is 51.57 at iteration 303
✅ Fitting complete.

Fit results
✅ Success: True
⏱️ Fitting time: 11.13 seconds
📏 Goodness-of-fit (reduced χ²): 51.57
📏 R-factor (Rf): 19.70%
📏 R-factor squared (Rf²): 30.20%
📏 Weighted R-factor (wR): 30.35%
📈 Fitted parameters:


,datablock,category,entry,parameter,start,fitted,uncertainty,units,change
1,hs,cell,,length_a,6.9000,6.8623,0.0003,Å,0.55 % ↓
2,hs,cell,,length_c,14.1000,14.1365,0.0008,Å,0.26 % ↑
3,hrpt,instrument,,twotheta_offset,0.0000,0.1271,0.0051,deg,N/A
4,hrpt,linked_phases,hs,scale,0.5000,0.2548,0.0030,,49.04 % ↓


#### Plot Measured vs Calculated

In [27]:
project.plot_meas_vs_calc(expt_name='hrpt', show_residual=True)

In [28]:
project.plot_meas_vs_calc(expt_name='hrpt', x_min=48, x_max=51, show_residual=True)

### Perform Fit 2/5

Set more parameters to be refined.

In [29]:
expt.peak.broad_gauss_u.free = True
expt.peak.broad_gauss_v.free = True
expt.peak.broad_gauss_w.free = True
expt.peak.broad_lorentz_x.free = True

for point in expt.background:
    point.y.free = True

Show free parameters after selection.

In [30]:
project.analysis.show_free_params()


Free parameters for both sample models (🧩 data blocks) and experiments (🔬 data blocks)


,datablock,category,entry,parameter,value,uncertainty,min,max,units
1,hs,cell,,length_a,6.8623,0.0003,,,Å
2,hs,cell,,length_c,14.1365,0.0008,,,Å
3,hrpt,background,4.4196,y,500.0000,,,,
4,hrpt,background,6.6207,y,500.0000,,,,
5,hrpt,background,10.4918,y,500.0000,,,,
6,hrpt,background,15.4634,y,500.0000,,,,
7,hrpt,background,45.6041,y,500.0000,,,,
8,hrpt,background,74.6844,y,500.0000,,,,
9,hrpt,background,103.4187,y,500.0000,,,,
10,hrpt,background,121.6311,y,500.0000,,,,


#### Run Fitting

In [31]:
project.analysis.fit()


Using experiment 🔬 'hrpt' for 'single' fitting
🚀 Starting fit process with 'lmfit (leastsq)'...
📈 Goodness-of-fit (reduced χ²) change:


iteration,χ²,improvement [%]
1,51.78,
21,13.07,74.8% ↓
40,12.66,3.2% ↓
58,12.49,1.3% ↓
340,12.41,


🏆 Best goodness-of-fit (reduced χ²) is 12.41 at iteration 339
✅ Fitting complete.



Fit results
✅ Success: True
⏱️ Fitting time: 11.93 seconds
📏 Goodness-of-fit (reduced χ²): 12.41
📏 R-factor (Rf): 9.62%
📏 R-factor squared (Rf²): 12.83%
📏 Weighted R-factor (wR): 12.21%
📈 Fitted parameters:


,datablock,category,entry,parameter,start,fitted,uncertainty,units,change
1,hs,cell,,length_a,6.8623,6.8628,0.0003,Å,0.01 % ↑
2,hs,cell,,length_c,14.1365,14.1389,0.0009,Å,0.02 % ↑
3,hrpt,background,4.4196,y,500.0000,645.4830,23.7674,,29.10 % ↑
4,hrpt,background,6.6207,y,500.0000,520.8595,13.6088,,4.17 % ↑
5,hrpt,background,10.4918,y,500.0000,455.0036,10.4220,,9.00 % ↓
6,hrpt,background,15.4634,y,500.0000,428.8639,5.4019,,14.23 % ↓
7,hrpt,background,45.6041,y,500.0000,453.5008,5.2135,,9.30 % ↓
8,hrpt,background,74.6844,y,500.0000,446.7592,5.4686,,10.65 % ↓
9,hrpt,background,103.4187,y,500.0000,411.5620,5.4490,,17.69 % ↓
10,hrpt,background,121.6311,y,500.0000,364.3828,5.6634,,27.12 % ↓


#### Plot Measured vs Calculated

In [32]:
project.plot_meas_vs_calc(expt_name='hrpt', show_residual=True)

In [33]:
project.plot_meas_vs_calc(expt_name='hrpt', x_min=48, x_max=51, show_residual=True)

### Perform Fit 3/5

Set more parameters to be refined.

In [34]:
model.atom_sites['O'].fract_x.free = True
model.atom_sites['O'].fract_z.free = True
model.atom_sites['Cl'].fract_z.free = True
model.atom_sites['H'].fract_x.free = True
model.atom_sites['H'].fract_z.free = True

Show free parameters after selection.

In [35]:
project.analysis.show_free_params()


Free parameters for both sample models (🧩 data blocks) and experiments (🔬 data blocks)


,datablock,category,entry,parameter,value,uncertainty,min,max,units
1,hs,atom_sites,O,fract_x,0.2100,,,,
2,hs,atom_sites,O,fract_z,0.0600,,,,
3,hs,atom_sites,Cl,fract_z,0.1970,,,,
4,hs,atom_sites,H,fract_x,0.1300,,,,
5,hs,atom_sites,H,fract_z,0.0800,,,,
6,hs,cell,,length_a,6.8628,0.0003,,,Å
7,hs,cell,,length_c,14.1389,0.0009,,,Å
8,hrpt,background,4.4196,y,645.4830,23.7674,,,
9,hrpt,background,6.6207,y,520.8595,13.6088,,,
10,hrpt,background,10.4918,y,455.0036,10.4220,,,


#### Run Fitting

In [36]:
project.analysis.fit()


Using experiment 🔬 'hrpt' for 'single' fitting
🚀 Starting fit process with 'lmfit (leastsq)'...
📈 Goodness-of-fit (reduced χ²) change:


iteration,χ²,improvement [%]
1,12.43,
26,5.11,58.9% ↓
49,4.40,13.9% ↓
72,4.35,1.3% ↓
188,4.34,


🏆 Best goodness-of-fit (reduced χ²) is 4.34 at iteration 187
✅ Fitting complete.



Fit results
✅ Success: True
⏱️ Fitting time: 6.00 seconds
📏 Goodness-of-fit (reduced χ²): 4.34
📏 R-factor (Rf): 6.01%
📏 R-factor squared (Rf²): 7.74%
📏 Weighted R-factor (wR): 7.51%
📈 Fitted parameters:


,datablock,category,entry,parameter,start,fitted,uncertainty,units,change
1,hs,atom_sites,O,fract_x,0.2100,0.2059,0.0002,,1.96 % ↓
2,hs,atom_sites,O,fract_z,0.0600,0.0625,0.0002,,4.14 % ↑
3,hs,atom_sites,Cl,fract_z,0.1970,0.1977,0.0002,,0.36 % ↑
4,hs,atom_sites,H,fract_x,0.1300,0.1330,0.0002,,2.33 % ↑
5,hs,atom_sites,H,fract_z,0.0800,0.0877,0.0001,,9.59 % ↑
6,hs,cell,,length_a,6.8628,6.8621,0.0002,Å,0.01 % ↓
7,hs,cell,,length_c,14.1389,14.1356,0.0005,Å,0.02 % ↓
8,hrpt,background,4.4196,y,645.4830,647.4024,14.0610,,0.30 % ↑
9,hrpt,background,6.6207,y,520.8595,523.4725,8.0510,,0.50 % ↑
10,hrpt,background,10.4918,y,455.0036,453.7545,6.1678,,0.27 % ↓


#### Plot Measured vs Calculated

In [37]:
project.plot_meas_vs_calc(expt_name='hrpt', show_residual=True)

In [38]:
project.plot_meas_vs_calc(expt_name='hrpt', x_min=48, x_max=51, show_residual=True)

### Perform Fit 4/5

Set more parameters to be refined.

In [39]:
model.atom_sites['Zn'].b_iso.free = True
model.atom_sites['Cu'].b_iso.free = True
model.atom_sites['O'].b_iso.free = True
model.atom_sites['Cl'].b_iso.free = True
model.atom_sites['H'].b_iso.free = True

Show free parameters after selection.

In [40]:
project.analysis.show_free_params()


Free parameters for both sample models (🧩 data blocks) and experiments (🔬 data blocks)


,datablock,category,entry,parameter,value,uncertainty,min,max,units
1,hs,atom_sites,Zn,b_iso,0.5000,,,,Å²
2,hs,atom_sites,Cu,b_iso,0.5000,,,,Å²
3,hs,atom_sites,O,b_iso,0.5000,,,,Å²
4,hs,atom_sites,O,fract_x,0.2059,0.0002,,,
5,hs,atom_sites,O,fract_z,0.0625,0.0002,,,
6,hs,atom_sites,Cl,b_iso,0.5000,,,,Å²
7,hs,atom_sites,Cl,fract_z,0.1977,0.0002,,,
8,hs,atom_sites,H,b_iso,0.5000,,,,Å²
9,hs,atom_sites,H,fract_x,0.1330,0.0002,,,
10,hs,atom_sites,H,fract_z,0.0877,0.0001,,,


#### Run Fitting

In [41]:
project.analysis.fit()


Using experiment 🔬 'hrpt' for 'single' fitting
🚀 Starting fit process with 'lmfit (leastsq)'...
📈 Goodness-of-fit (reduced χ²) change:


iteration,χ²,improvement [%]
1,4.35,
31,2.30,47.1% ↓
59,2.11,8.3% ↓
144,2.11,


🏆 Best goodness-of-fit (reduced χ²) is 2.11 at iteration 116
✅ Fitting complete.



Fit results
✅ Success: True
⏱️ Fitting time: 5.40 seconds
📏 Goodness-of-fit (reduced χ²): 2.11
📏 R-factor (Rf): 4.17%
📏 R-factor squared (Rf²): 5.05%
📏 Weighted R-factor (wR): 4.73%
📈 Fitted parameters:


,datablock,category,entry,parameter,start,fitted,uncertainty,units,change
1,hs,atom_sites,Zn,b_iso,0.5000,0.0855,0.0630,Å²,82.91 % ↓
2,hs,atom_sites,Cu,b_iso,0.5000,1.1937,0.0399,Å²,138.74 % ↑
3,hs,atom_sites,O,b_iso,0.5000,0.7001,0.0360,Å²,40.02 % ↑
4,hs,atom_sites,O,fract_x,0.2059,0.2060,0.0002,,0.06 % ↑
5,hs,atom_sites,O,fract_z,0.0625,0.0609,0.0001,,2.53 % ↓
6,hs,atom_sites,Cl,b_iso,0.5000,1.1129,0.0385,Å²,122.58 % ↑
7,hs,atom_sites,Cl,fract_z,0.1977,0.1968,0.0001,,0.49 % ↓
8,hs,atom_sites,H,b_iso,0.5000,2.3404,0.0401,Å²,368.08 % ↑
9,hs,atom_sites,H,fract_x,0.1330,0.1322,0.0002,,0.62 % ↓
10,hs,atom_sites,H,fract_z,0.0877,0.0900,0.0001,,2.63 % ↑


#### Plot Measured vs Calculated

In [42]:
project.plot_meas_vs_calc(expt_name='hrpt', show_residual=True)

In [43]:
project.plot_meas_vs_calc(expt_name='hrpt', x_min=48, x_max=51, show_residual=True)

## Summary

This final section shows how to review the results of the analysis.

#### Show Project Summary

In [44]:
project.summary.show_report()


*** PROJECT INFO ***

Title
Untitled Project

*** CRYSTALLOGRAPHIC DATA ***

Phase datablock
🧩 hs

Space group
R -3 m

Cell parameters


alpha,90.00000
beta,90.00000
gamma,120.00000
a,6.86149
b,6.86149
c,14.13604



Atom sites


Label,Type,fract_x,fract_y,fract_z,Occupancy,B_iso
Zn,Zn,0.00000,0.00000,0.50000,1.00000,0.08546
Cu,Cu,0.50000,0.00000,0.00000,1.00000,1.19372
O,O,0.20601,-0.20601,0.06090,1.00000,0.70009
Cl,Cl,0.00000,0.00000,0.19675,1.00000,1.11288
H,2H,0.13220,-0.13220,0.08998,1.00000,2.34039



*** EXPERIMENTS ***

Experiment datablock
🔬 hrpt

Experiment type
powder, neutron, constant wavelength

Wavelength
1.89000

2θ offset
0.11368

Profile type
pseudo-voigt

Peak broadening (Gaussian)


U,0.15795
V,-0.35708
W,0.34978



Peak broadening (Lorentzian)


X,0.29266
Y,0.00000



*** FITTING ***

Calculation engine
cryspy

Minimization engine
lmfit (leastsq)

Fit quality


Goodness-of-fit (reduced χ²),2.11
